In [ ]:
# import libraries
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# set target image file
image_filename = 'target_image.png'

In [ ]:
# define function to calculate Fourier transform of the grating of the given image
def calculate_2dft(input):
    ft = np.fft.ifftshift(input)
    ft = np.fft.fft2(ft)
    return np.fft.fftshift(ft)

# define function to calculate the inverse Fourier transform of the Fourier transform of the given image
def calculate_2dift(input):
    ift = np.fft.ifftshift(input)
    ift = np.fft.ifft2(ift)
    ift = np.fft.fftshift(ift)
    return ift.real

# define function to calculate distance from given coordinates to center
def calculate_distance_from_centre(coords, centre):
    return np.sqrt(
        (coords[0] - centre) ** 2 + (coords[1] - centre) ** 2
    )

# define function to find coordinates symmetric to given coordinates
def find_symmetric_coordinates(coords, centre):
    return (centre + (centre - coords[0]),
            centre + (centre - coords[1]))

# define function to display plots
def display_plots(individual_grating, reconstruction, idx):
    plt.subplot(121)
    plt.imshow(individual_grating)
    plt.axis('off')
    plt.subplot(122)
    plt.imshow(reconstruction)
    plt.axis('off')
    plt.suptitle(f'Terms: {idx}')
    #plt.pause(0.01)

In [ ]:
# read image
image = plt.imread(image_filename)
image = image[:, :, :3].mean(axis = 2)

# set array dimensions and center array
array_size = min(image.shape) - 1 + min(image.shape) % 2
image = image[:array_size, :array_size]
centre = int((array_size - 1) / 2)

# identify coordinate pairs on the left side of the array
coords_left_half = (
    (x, y) for x in range(array_size) for y in range(centre + 1)
)

# sort coordinate pairs in order of distance from center
coords_left_half = sorted(
    coords_left_half,
    key = lambda x: calculate_distance_from_centre(x, centre)
)

plt.set_cmap('gray')

ft = calculate_2dft(image)

# display grayscale image and its Fourier transform
plt.subplot(121)
plt.imshow(image)
plt.axis('off')
plt.subplot(122)
plt.imshow(np.log(abs(ft)))
plt.axis('off')
plt.pause(2)

# display image reconstruction
fig = plt.figure()

# generate blank arrays
rec_image = np.zeros(image.shape)
individual_grating = np.zeros(
    image.shape, dtype = 'complex'
)
idx = 0

# set value until which all steps are displayed
display_all_until = 200
# set value to skip steps to display
display_step = 10
# set index of next step to display
next_display = display_all_until + display_step

file_number = 0
for coords in coords_left_half:
    # only include if points are in the top half of the center column
    if not (coords[1] == centre and coords[0] > centre):
        idx += 1
        symm_coords = find_symmetric_coordinates(
            coords, centre
        )
        
        # Copy values from Fourier transform for the given pair of points
        individual_grating[coords] = ft[coords]
        individual_grating[symm_coords] = ft[symm_coords]
               
        # Calculate inverse Fourier transform and add this to the reconstructed image
        rec_grating = calculate_2dift(individual_grating)
        rec_image += rec_grating  
        
        # Clear the array
        individual_grating[coords] = 0
        individual_grating[symm_coords] = 0        

        # display steps using step display index set above
        if idx < display_all_until or idx == next_display:
            if idx > display_all_until:
                next_display += display_step
                display_step += 10
            display_plots(rec_grating, rec_image, idx)
            print(f'Saving image {file_number + 1}')
            plt.savefig(f'images\image_{file_number}.png')
            file_number += 1